In [1]:
import os

from langgraph.graph import StateGraph, START, END


from Nodes.generate_component import *
from Nodes.generate_layout import *
from Nodes.generate_kpi import *

# os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("CUSTOM_TRACING_V2", "true")
# os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("CUSTOM_ENDPOINT", "https://api.smith.langchain.com")
# os.environ["LANGCHAIN_PROJECT"] = os.getenv("CUSTOM_PROJECT", "CFOLytics_reportgenerator")

##########################################
# Build the specialized component subgraph
##########################################
component_subgraph = StateGraph(SpecializedComponentState)

component_subgraph.add_node("component_selector", component_selector)
component_subgraph.add_node("generate_chart_component", generate_chart_component)
component_subgraph.add_node("generate_table_component", generate_table_component)
component_subgraph.add_node("generate_waterfall_component", generate_waterfall_component)
component_subgraph.add_node("generate_tile_component", generate_tile_component)
component_subgraph.add_node("generate_generic_component", generate_generic_component)

component_subgraph.add_edge(START, "component_selector")

component_subgraph.add_conditional_edges(
    "component_selector",
    selector_routing,
    [
        "generate_chart_component",
        "generate_table_component",
        "generate_waterfall_component",
        "generate_tile_component",
        "generate_generic_component"
    ]
)

# Each specialized node ends immediately
component_subgraph.add_edge("generate_chart_component", END)
component_subgraph.add_edge("generate_table_component", END)
component_subgraph.add_edge("generate_waterfall_component", END)
component_subgraph.add_edge("generate_tile_component", END)
component_subgraph.add_edge("generate_generic_component", END)

# Compile subgraph
generate_component_subgraph = component_subgraph.compile()

from Nodes.mainchart_helper_functions import *


from Classes.state_classes import ListSubgraphState, KpiSubgraphState
from Nodes.generate_list import *

#########################################################
# Build the KPI generation subgraph
#########################################################
kpi_subgraph = StateGraph(KpiSubgraphState)

kpi_subgraph.add_node("analyze_kpi_dimensions", analyze_kpi_dimensions)
kpi_subgraph.add_node("resolve_dimension_members", resolve_dimension_members)
kpi_subgraph.add_node("create_kpi_configuration", create_kpi_configuration)

kpi_subgraph.add_edge(START, "analyze_kpi_dimensions")
kpi_subgraph.add_edge("analyze_kpi_dimensions", "resolve_dimension_members")
kpi_subgraph.add_edge("resolve_dimension_members", "create_kpi_configuration")
kpi_subgraph.add_edge("create_kpi_configuration", END)

# Compile the KPI subgraph
generate_kpi_subgraph = kpi_subgraph.compile()

# Process function for KPIs that handles all KPIs at once
def process_kpi_directly(state):
    """
    Process all KPIs directly in the main graph.
    This simulates running the KPI subgraph for all KPIs at once.
    """
    # Skip if no KPIs to process
    kpis = state.get("Kpis", [])
    if not kpis:
        return state
        
    # Initialize JsonKpis if not present
    if "JsonKpis" not in state:
        state["JsonKpis"] = []
        
    # Process all KPIs
    for kpi_description in kpis:
        # Create a KPI state that would normally be used by the subgraph
        kpi_state = {
            "Kpi": kpi_description,
            "dimensions": [],
            "dimensionHierarchies": {},
            "ReportMetadata": state.get("ReportMetadata", []),
            "JsonKpis": []
        }
        
        # Run the KPI subgraph logic directly on each KPI
        kpi_state = analyze_kpi_dimensions(kpi_state)
        kpi_state = resolve_dimension_members(kpi_state)
        kpi_state = create_kpi_configuration(kpi_state)
        
        # Add the generated KPI to the main state
        state["JsonKpis"].extend(kpi_state.get("JsonKpis", []))
    
    return state

#########################################################
# Build the list generation subgraph
#########################################################
list_subgraph = StateGraph(ListSubgraphState)

list_subgraph.add_node("check_if_list_exists", check_if_list_exists)
list_subgraph.add_node("return_existing_list", return_existing_list)
list_subgraph.add_node("check_dynamic_or_fixed", check_dynamic_or_fixed)
list_subgraph.add_node("create_fixed_list", create_fixed_list)
list_subgraph.add_node("create_dynamic_list", create_dynamic_list)

list_subgraph.add_edge(START, "check_if_list_exists")


list_subgraph.add_conditional_edges("check_if_list_exists", list_exists_routing,
                               ["return_existing_list", "check_dynamic_or_fixed"])
list_subgraph.add_edge("return_existing_list", END)


list_subgraph.add_conditional_edges("check_dynamic_or_fixed", dynamic_or_fixed_routing,
                               ["create_dynamic_list", "create_fixed_list"])
list_subgraph.add_edge("create_dynamic_list", END)
list_subgraph.add_edge("create_fixed_list", END)

# Compile the list subgraph
generate_list_subgraph = list_subgraph.compile()

# Define a function for handling the end of list processing
def continue_after_lists(state):
    """Decide where to go after list processing."""
    return ["consolidate_lists_to_layout"]

##########################
# MAIN GRAPH
##########################
graph = StateGraph(OverallState)

# Main flow nodes
graph.add_node("generate_conceptualdesign", generate_conceptualdesign)
graph.add_node("generate_layout", generate_layout)
graph.add_node("generate_component_subgraph", generate_component_subgraph)
graph.add_node("update_json_layout", update_json_layout)

# KPI and list processing nodes
graph.add_node("gatheruniquekpis", gatheruniquekpis)
graph.add_node("generate_kpi_subgraph", process_kpi_directly)
graph.add_node("consolidate_kpis_to_layout", consolidate_kpis_to_layout)
graph.add_node("gatheruniquelists", gatheruniquelists)
graph.add_node("generate_list_subgraph", generate_list_subgraph)
graph.add_node("consolidate_lists_to_layout", consolidate_lists_to_layout)

graph.add_edge(START, "generate_conceptualdesign")
graph.add_edge("generate_conceptualdesign", "generate_layout")

# After we have the base layout and extracted all 'Components',
# route each component to the specialized subgraph
graph.add_conditional_edges("generate_layout", continue_to_components, ["generate_component_subgraph"])

# Then from the specialized subgraph, we go to update_json_layout
graph.add_edge("generate_component_subgraph", "update_json_layout")

# Extract KPIs from components
graph.add_edge("update_json_layout", "gatheruniquekpis")

# KPI Processing - fully conditional 
graph.add_conditional_edges("gatheruniquekpis", continue_to_kpis, 
                           ["generate_kpi_subgraph", "gatheruniquelists"])
graph.add_edge("generate_kpi_subgraph", "consolidate_kpis_to_layout")
graph.add_edge("consolidate_kpis_to_layout", "gatheruniquelists")

# List processing flow - fully conditional
graph.add_conditional_edges("gatheruniquelists", continue_to_lists, 
                           ["generate_list_subgraph", "consolidate_lists_to_layout"])
graph.add_edge("generate_list_subgraph", "consolidate_lists_to_layout")
graph.add_edge("consolidate_lists_to_layout", END)

# Compile the main graph
app = graph.compile()

In [2]:
from IPython.display import Image
Image(app.get_graph().draw_mermaid_png())


ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [ ]:
import json
from typing import Annotated, List


with open("sample.json", "r", encoding="utf-8") as f:
    state_data = json.load(f)
    state = OverallState(state_data)
    result_state = app.invoke(state)

final_json = json.dumps(result_state["JsonLayoutList"], ensure_ascii=False, separators=(",", ":")).replace("True", "true").replace("False", "false")

print(final_json)


KeyboardInterrupt: 